In [1]:
import dbt
import json
import IPython.display as ipd
import utilities
import Segment
import Alignment
import librosa

In [10]:
song1_name = "demon_beat.wav"

In [5]:
print(len(song1_data) / sr)

237.68816326530612


In [11]:
json_song1_name = song1_name.replace(".wav", "") + ".json"
with open(json_song1_name, "w") as json_file:
    dump_data = []
#     song1_segments = Segment.extract_segment(segmentino_name)
    song_data, sr  = dbt.load_and_resample(song1_name)
    song_data = librosa.effects.harmonic(song_data)
    song1_segments = [Segment.Segment(0, len(song_data) / sr, "")]
    for idx in range(len(song1_segments)):
#         song_data,sr = dbt.load_and_resample(song1_name.replace(".wav", "") + "_" + str(idx) + ".wav")
        print("Loaded song_data with sr", sr, "length", len(song_data) / sr)
        print("Segment info:", song1_segments[idx].toString(), "idx", idx)    
        #Get Downbeats
        downbeats = dbt.extract_downbeat(4,song_data)
        
        time_1, beat_num1 = downbeats[:,0].tolist(), downbeats[:,1].tolist()
#         if time_1[0] > 0.07:
#             time_1 = [0.0] + time_1
#             beat_num1 = [(beat_num1[0] - 1) % 4] + beat_num1
        
        print("length time_1", len(time_1))
        #Get Chord
        chord_dict, chord_result1 = dbt.get_chords_old(time_1, song_data, sr, False)
        #Smoothen the chords
        print("Chord_result",chord_result1)
        new_chord_result, isAligned = Alignment.chord_alignment(chord_result1)
        song1_segments[idx].useable = isAligned
        
        #Squash the same beats together
        if isAligned:
            boolean_array = [False for i in range(len(new_chord_result))]
            for i in range(len(new_chord_result)):
                if i == 0 or new_chord_result[i] != new_chord_result[i-1]:
                    boolean_array[i] = True
            new_chords = []
            new_time = []
            for i in range(len(boolean_array)):
                if boolean_array[i]:
                    new_chords.append(new_chord_result[i])
                    new_time.append(time_1[i])
            new_chord_result = new_chords
            time_1 = new_time
            
        #Average time per beat
        avg_time_beat = 0
        for i in range(len(time_1) - 1):
            avg_time_beat += time_1[i+1] - time_1[i]
        avg_time_beat = avg_time_beat / len(time_1)
        print("Average time_per_chord", avg_time_beat)
        
        if not isAligned:
            print("Cannot align chord")
        
        song1_segments[idx].chords = new_chord_result
        song1_segments[idx].downbeats = time_1
        song1_segments[idx].average_time_per_beat = avg_time_beat
        
        
        beat_num1 = [1 for i in range(len(time_1))]
        
        dbt.print_and_save_chord(song1_name, time_1, song1_segments[idx].chords, str(idx) )
        dbt.print_and_save_downbeat(song1_name, time_1, beat_num1, str(idx))
        
        dump_data.append(song1_segments[idx].getJSON())    
    json.dump(dump_data, json_file, indent=4)
        

Loaded song_data with sr 44100 length 183.80916099773242
Segment info: Start:0 End:183.80916099773242 Duration: 183.80916099773242 Type: downbeat_length:0 chord_length:0 idx 0
length time_1 259
index 0 : 0.76 : 1.69 duration: 0.9299999999999999 [(0., 1., 'N')]
index 1 : 1.69 : 2.61 duration: 0.9199999999999999 [(0., 1., 'N')]
index 2 : 2.61 : 3.6 duration: 0.9900000000000002 [(0., 1., 'N')]
index 3 : 3.6 : 4.5 duration: 0.8999999999999999 [(0., 0.9, 'N')]
index 4 : 4.5 : 5.43 duration: 0.9299999999999997 [(0., 1., 'N')]
index 5 : 5.43 : 6.15 duration: 0.7200000000000006 [(0., 0.8, 'N')]
index 6 : 6.15 : 6.81 duration: 0.6599999999999993 [(0., 0.7, 'N')]
index 7 : 6.81 : 7.49 duration: 0.6800000000000006 [(0., 0.7, 'D#:maj')]
index 8 : 7.49 : 8.14 duration: 0.6500000000000004 [(0., 0.7, 'D#:maj')]
index 9 : 8.14 : 8.81 duration: 0.6699999999999999 [(0., 0.7, 'D#:maj')]
index 10 : 8.81 : 9.48 duration: 0.6699999999999999 [(0., 0.7, 'D#:maj')]
index 11 : 9.48 : 10.14 duration: 0.660000000

index 109 : 74.82 : 75.48 duration: 0.6600000000000108 [(0., 0.7, 'A#:maj')]
index 110 : 75.48 : 76.14 duration: 0.6599999999999966 [(0., 0.7, 'C:min')]
index 111 : 76.14 : 76.81 duration: 0.6700000000000017 [(0., 0.7, 'C:min')]
index 112 : 76.81 : 77.48 duration: 0.6700000000000017 [(0., 0.7, 'C:min')]
index 113 : 77.48 : 78.15 duration: 0.6700000000000017 [(0., 0.7, 'C:min')]
index 114 : 78.15 : 78.81 duration: 0.6599999999999966 [(0., 0.7, 'G#:maj')]
index 115 : 78.81 : 79.48 duration: 0.6700000000000017 [(0., 0.7, 'G#:maj')]
index 116 : 79.48 : 80.15 duration: 0.6700000000000017 [(0., 0.7, 'G#:maj')]
index 117 : 80.15 : 80.82 duration: 0.6699999999999875 [(0., 0.7, 'G#:maj')]
index 118 : 80.82 : 81.48 duration: 0.6600000000000108 [(0., 0.7, 'G#:maj')]
index 119 : 81.48 : 82.15 duration: 0.6700000000000017 [(0., 0.7, 'D#:maj')]
index 120 : 82.15 : 82.81 duration: 0.6599999999999966 [(0., 0.7, 'D#:maj')]
index 121 : 82.81 : 83.48 duration: 0.6700000000000017 [(0., 0.7, 'D#:maj')]
ind

index 220 : 150.53 : 151.48 duration: 0.9499999999999886 [(0., 1., 'C:min')]
index 221 : 151.48 : 152.3 duration: 0.8200000000000216 [(0., 0.9, 'C:min')]
index 222 : 152.3 : 153.19 duration: 0.8899999999999864 [(0., 0.9, 'C:min')]
index 223 : 153.19 : 153.97 duration: 0.7800000000000011 [(0., 0.8, 'N')]
index 224 : 153.97 : 154.73 duration: 0.7599999999999909 [(0., 0.8, 'N')]
index 225 : 154.73 : 155.46 duration: 0.7300000000000182 [(0., 0.8, 'N')]
index 226 : 155.46 : 156.16 duration: 0.6999999999999886 [(0., 0.7, 'D#:maj')]
index 227 : 156.16 : 156.8 duration: 0.6400000000000148 [(0., 0.7, 'D#:maj')]
index 228 : 156.8 : 157.49 duration: 0.6899999999999977 [(0., 0.7, 'D#:maj')]
index 229 : 157.49 : 158.14 duration: 0.6499999999999773 [(0., 0.7, 'D#:maj')]
index 230 : 158.14 : 158.82 duration: 0.6800000000000068 [(0., 0.7, 'A#:maj')]
index 231 : 158.82 : 159.47 duration: 0.6500000000000057 [(0., 0.7, 'A#:maj')]
index 232 : 159.47 : 160.15 duration: 0.6800000000000068 [(0., 0.7, 'A#:maj

In [9]:
#Raw, debug
sample,sr = dbt.load_and_resample(song1_name)
downbeats = dbt.extract_downbeat(4,sample)
dbt.print_and_save_downbeat(song1_name, downbeats[:,0], downbeats[:,1])
time_1, beat_num1 = downbeats[:,0].tolist(), downbeats[:,1].tolist()
# time_1, beat_num1 = dbt.filter_downbeat(downbeats)
# time_1 = downbeats[:,0]
chord_dict, chord_result = dbt.get_chords_old(time_1, sample, sr)
dbt.print_and_save_chord(song1_name, downbeats[:,0], chord_result)



Saving downbeat of faded.wav
index 0 : 0.61 : 1.28 duration: 0.67 [(0., 0.7, 'D#:min')]
index 1 : 1.28 : 1.96 duration: 0.6799999999999999 [(0., 0.7, 'F#:maj')]
index 2 : 1.96 : 2.62 duration: 0.6600000000000001 [(0., 0.7, 'D#:min')]
index 3 : 2.62 : 3.28 duration: 0.6599999999999997 [(0., 0.7, 'F#:maj')]
index 4 : 3.28 : 3.95 duration: 0.6700000000000004 [(0., 0.7, 'B:maj')]
index 5 : 3.95 : 4.62 duration: 0.6699999999999999 [(0., 0.7, 'B:maj')]
index 6 : 4.62 : 5.28 duration: 0.6600000000000001 [(0., 0.7, 'B:maj')]
index 7 : 5.28 : 5.94 duration: 0.6600000000000001 [(0., 0.7, 'B:maj')]
index 8 : 5.94 : 6.62 duration: 0.6799999999999997 [(0., 0.7, 'F#:maj')]
index 9 : 6.62 : 7.28 duration: 0.6600000000000001 [(0., 0.7, 'F#:maj')]
index 10 : 7.28 : 7.95 duration: 0.6699999999999999 [(0., 0.7, 'F#:maj')]
index 11 : 7.95 : 8.62 duration: 0.669999999999999 [(0., 0.7, 'F#:maj')]
index 12 : 8.62 : 9.28 duration: 0.6600000000000001 [(0., 0.7, 'C#:maj')]
index 13 : 9.28 : 9.96 duration: 0.680

index 111 : 75.98 : 76.61 duration: 0.6299999999999955 [(0., 0.7, 'C#:maj')]
index 112 : 76.61 : 77.28 duration: 0.6700000000000017 [(0., 0.7, 'D#:min')]
index 113 : 77.28 : 77.94 duration: 0.6599999999999966 [(0., 0.7, 'D#:min')]
index 114 : 77.94 : 78.61 duration: 0.6700000000000017 [(0., 0.7, 'D#:maj')]
index 115 : 78.61 : 79.28 duration: 0.6700000000000017 [(0., 0.7, 'D#:maj')]
index 116 : 79.28 : 79.94 duration: 0.6599999999999966 [(0., 0.7, 'B:maj')]
index 117 : 79.94 : 80.61 duration: 0.6700000000000017 [(0., 0.7, 'B:maj')]
index 118 : 80.61 : 81.28 duration: 0.6700000000000017 [(0., 0.7, 'B:maj')]
index 119 : 81.28 : 81.94 duration: 0.6599999999999966 [(0., 0.7, 'B:maj')]
index 120 : 81.94 : 82.61 duration: 0.6700000000000017 [(0., 0.7, 'F#:maj')]
index 121 : 82.61 : 83.28 duration: 0.6700000000000017 [(0., 0.7, 'F#:maj')]
index 122 : 83.28 : 83.94 duration: 0.6599999999999966 [(0., 0.7, 'F#:maj')]
index 123 : 83.94 : 84.61 duration: 0.6700000000000017 [(0., 0.7, 'F#:maj')]
ind

index 222 : 122.94 : 123.28 duration: 0.3400000000000034 [(0., 0.4, 'N')]
index 223 : 123.28 : 123.61 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 224 : 123.61 : 123.95 duration: 0.3400000000000034 [(0., 0.4, 'N')]
index 225 : 123.95 : 124.28 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 226 : 124.28 : 124.61 duration: 0.3299999999999983 [(0., 0.4, 'N')]
index 227 : 124.61 : 124.95 duration: 0.3400000000000034 [(0., 0.4, 'N')]
index 228 : 124.95 : 125.28 duration: 0.3299999999999983 [(0., 0.4, 'F#:maj')]
index 229 : 125.28 : 125.61 duration: 0.3299999999999983 [(0., 0.4, 'F#:maj')]
index 230 : 125.61 : 125.94 duration: 0.3299999999999983 [(0., 0.4, 'F#:maj')]
index 231 : 125.94 : 126.28 duration: 0.3400000000000034 [(0., 0.4, 'F#:maj')]
index 232 : 126.28 : 126.61 duration: 0.3299999999999983 [(0., 0.4, 'F#:maj')]
index 233 : 126.61 : 126.94 duration: 0.3299999999999983 [(0., 0.4, 'F#:maj')]
index 234 : 126.94 : 127.28 duration: 0.3400000000000034 [(0., 0.4, 'F#:maj')]
ind

index 332 : 167.28 : 167.94 duration: 0.6599999999999966 [(0., 0.7, 'B:maj')]
index 333 : 167.94 : 168.61 duration: 0.6700000000000159 [(0., 0.7, 'B:maj')]
index 334 : 168.61 : 169.28 duration: 0.6699999999999875 [(0., 0.7, 'F#:maj')]
index 335 : 169.28 : 169.94 duration: 0.6599999999999966 [(0., 0.7, 'F#:maj')]
index 336 : 169.94 : 170.61 duration: 0.6700000000000159 [(0., 0.7, 'F#:maj')]
index 337 : 170.61 : 171.28 duration: 0.6699999999999875 [(0., 0.7, 'F#:maj')]
index 338 : 171.28 : 171.94 duration: 0.6599999999999966 [(0., 0.7, 'C#:maj')]
index 339 : 171.94 : 172.6 duration: 0.6599999999999966 [(0., 0.7, 'C#:maj')]
index 340 : 172.6 : 173.31 duration: 0.710000000000008 [(0., 0.8, 'C#:maj')]
index 341 : 173.31 : 173.94 duration: 0.6299999999999955 [(0., 0.7, 'C#:maj')]
index 342 : 173.94 : 174.61 duration: 0.6700000000000159 [(0., 0.7, 'D#:min')]
index 343 : 174.61 : 175.28 duration: 0.6699999999999875 [(0., 0.7, 'D#:maj')]
index 344 : 175.28 : 175.94 duration: 0.6599999999999966 

In [6]:
print(downbeats)

[[  0.31   1.  ]
 [  0.81   2.  ]
 [  1.34   3.  ]
 [  1.83   4.  ]
 [  2.33   1.  ]
 [  2.85   2.  ]
 [  3.35   3.  ]
 [  3.86   4.  ]
 [  4.38   1.  ]
 [  4.9    2.  ]
 [  5.4    3.  ]
 [  5.9    4.  ]
 [  6.4    1.  ]
 [  6.91   2.  ]
 [  7.42   3.  ]
 [  7.92   4.  ]
 [  8.43   1.  ]
 [  8.94   2.  ]
 [  9.45   3.  ]
 [  9.96   4.  ]
 [ 10.47   1.  ]
 [ 10.97   2.  ]
 [ 11.48   3.  ]
 [ 12.     4.  ]
 [ 12.5    1.  ]
 [ 13.01   2.  ]
 [ 13.51   3.  ]
 [ 14.03   4.  ]
 [ 14.54   1.  ]
 [ 15.04   2.  ]
 [ 15.55   3.  ]
 [ 16.06   4.  ]
 [ 16.57   1.  ]
 [ 17.08   2.  ]
 [ 17.58   3.  ]
 [ 18.1    4.  ]
 [ 18.6    1.  ]
 [ 19.11   2.  ]
 [ 19.62   3.  ]
 [ 20.13   4.  ]
 [ 20.64   1.  ]
 [ 21.14   2.  ]
 [ 21.65   3.  ]
 [ 22.16   4.  ]
 [ 22.67   1.  ]
 [ 23.18   2.  ]
 [ 23.68   3.  ]
 [ 24.2    4.  ]
 [ 24.7    1.  ]
 [ 25.21   2.  ]
 [ 25.72   3.  ]
 [ 26.23   4.  ]
 [ 26.74   1.  ]
 [ 27.25   2.  ]
 [ 27.75   3.  ]
 [ 28.27   4.  ]
 [ 28.77   1.  ]
 [ 29.28   2.  ]
 [ 29.79   3. 